In [18]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from joblib import Parallel, delayed
from sklearn.metrics import mean_absolute_error
%matplotlib inline 
import numpy as np
plt.rcParams['figure.figsize'] = (16, 4)
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings(action="ignore")
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

from utils import * # We've added all the functions from the last BLU to the utils.py 
import seaborn as sns

import numpy as np
from random import gauss
from random import seed
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa import stattools
from pandas.plotting import lag_plot
from statsmodels.tsa.arima_model import ARIMA
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 4)



# Read and Import all Data

In [27]:
###train and test must be in the form of sereies so if we have a data frame we need to choose a column of it
# concatenate train and test
#df_ = pd.concat([train, test], axis=0)
#read data
dataset = 'dataset name'
#df_ = pd.read_csv(os.path.join('data',dataset + ".csv"))  
df_ = pd.read_csv('data/bitcoin.csv')
df = df_.copy()


df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d', infer_datetime_format=True)
df = df.set_index('Timestamp')
df = df.sort_index()
df
# then seperate train and test set

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2017-01-01 00:00:00,973.37,973.37,973.35,973.35,2.122048,2065.524303,973.363509
2017-01-01 00:01:00,973.37,973.37,973.35,973.35,2.122048,2065.524303,973.363509
2017-01-01 00:02:00,973.37,973.37,973.35,973.35,2.122048,2065.524303,973.363509
2017-01-01 00:03:00,973.36,973.36,973.36,973.36,0.040000,38.934400,973.360000
2017-01-01 00:04:00,973.36,973.40,973.36,973.39,5.458800,5313.529708,973.387871
...,...,...,...,...,...,...,...
2018-03-26 23:56:00,8155.00,8155.00,8154.99,8154.99,0.617945,5039.342643,8154.997667
2018-03-26 23:57:00,8154.99,8154.99,8154.00,8154.01,40.655410,331543.193980,8154.958865
2018-03-26 23:58:00,8154.00,8154.01,8150.00,8150.00,9.856911,80340.432933,8150.670628


# Data Preprocessing(Feature engineering)

In [ ]:
#filling the missing dates with zero and choosing one column to have series
df = df.resample('MS').asfreq()#asfreq() put all the new rows nan value
df_resampled = df.resample('D').mean()   #can be 'M','D','Y','W' or even df.resample('5 min').sum().plot()
data.isnull().sum()
df_cleaned = df_resampled.fillna(0)
df_cleaned = df_cleaned.customers
###filling the missing value with the method of forward fill
df['col with nans'].fillna(method='ffill').plot() #we can plot it also
#filling the missing value with KNN imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer() # To obtain better results we may want to have more neighbors, of course that in a real setting
                       # this is a parameter that should be tunned
    
data_imputed = imputer.fit_transform(df) # This may take a bit to run
df_imputed = pd.DataFrame(data_imputed, index=df.index, columns=df.columns)
df_imputed.'col with nans'.interpolate().plot() # we can plot it


In [ ]:
#adding day weekday etc to data frame
#df['day'] = df['date'].dt.day
#df['month'] = df['date'].dt.month
#df['year'] = df['date'].dt.year
#df['hour'] = df['date'].dt.hour
#df['minute'] = df['date'].dt.minute
#df['second'] = df['date'].dt.second
#df['day of the week'] = df['date'].dt.weekday
#df['day of the week name'] = df['date'].dt.day_name()
#df['quarter'] = df['date'].dt.quarter
#df['is it a leap year?'] = df['date'].dt.is_leap_year

#compute comulative sum

#df['a column name'].cumsum().plot();  # the total volume traded since the start 


In [29]:
#some question
#What was the weekly change in price, over time?#

# Feature engineering (adding more features to  data frame)

In [ ]:
#diff
df_cleaned['diff'] = df_cleaned.diff()
#Rolling windows
df_cleaned['rolling_max'] = df_cleaned['customers(some columns)'].rolling('7D').max()
df_cleaned['rolling_min'] = df_cleaned['customers(some columns)'].rolling('7D').min()
df_cleaned['rolling_mean'] = df_cleaned['customers(some columns)'].rolling('7D').mean()
df_cleaned['rolling_std'] = df_cleaned['customers(some columns)'].rolling('7D').std()
#add Holiday if it is 1 and if it is not 0
holidays = df_cleaned[((df_cleaned.index.month==12) & (df_cleaned.index.day==25))
              |((df_cleaned.index.month==1) & (df_cleaned.index.day==1))].customers
df_cleaned['holidays'] = holidays + 1
df_cleaned['holidays'] = df_cleaned['holidays'].fillna(0)
#day of week and month
df_cleaned['day_of_week'] = df_cleaned.index.weekday
df_cleaned['month'] = df_cleaned.index.month
#Circular encoding

df_cleaned['sin_weekday'] = np.sin(2*np.pi*df_cleaned.index.weekday/7)
df_cleaned['cos_weekday'] = np.cos(2*np.pi*df_cleaned.index.weekday/7)
        
df_cleaned['sin_month'] = np.sin(2*np.pi*df_cleaned.index.month/12)
df_cleaned['cos_month'] = np.cos(2*np.pi*df_cleaned.index.month/12)
df_cleaned = df_cleaned.drop(['day_of_week','month'], axis=1)



# If we have multi index

In [ ]:

data.set_index(['col1','col2'])
data.sort_index()
#group by multi index
food.groupby(level='Product').Price.idxmin()
#selecting from multi index
#Selecting from multi-index: dataframe.loc[idx[index_level_0, index_level_1], columns]
idx = pd.IndexSlice
data.loc[idx[:,'msft'],:]
#how to get indexes
data.loc[data.index.get_level_values(0).day_name()=='Monday'].Close.mean()
#percentage change
data.sort_index(level=['Stock', 'Date']).Close.pct_change().head()
# group by the stock, and then take the Close, and calculate percentage change 
data.groupby(level='Stock').Close.pct_change().head()
#Put one of the multi-index level into columns: unstack()
# group by using lambda function
food.groupby(level='Product').Price.agg(lambda x: list(x.loc[x == x.min()].index))

# Scatter plot

In [ ]:
# scatter plot of one column with respect to another one
plt.scatter(data['thousands of passengers'], data.lag_1)

# Correlation between one coulumns with its lags

In [ ]:
# It needs resample .asfriq in advance to fill the missing date with nans and then those nans will be ignored by corr function
data.corr()['thousands of passengers(it is a column of df)']

# Seasonal decomposition in time series

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
#To choose modell aditive or multiplicetive we have to plot the follwing:
df.rolling(24).std().plot();#24 is seasonal.It can be different
#If variance increasing model = Multiplicative if variance is constant model = additive
decomposition = seasonal_decompose(df, model='multiplicative')
decomposition.plot()
#To see the head of seasonal , trend , ... data
decomposition.trend.head(10)
decomposition.seasonal.head(10)


# ACF

In [ ]:
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
acf(df)


# Plot ACF

In [ ]:
# with 95% confidence interval.(this is beacause of alpha = 0.5)
plot_acf(df, alpha=.05,lags=50)
plt.xlabel('lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot PACF

In [ ]:
plot_pacf(df, alpha=0.05, lags=50, method='ols')
plt.xlabel('lag')
plt.ylabel('Autocorrelation')
plt.show()

# A stationary time series

In [ ]:
#mean, variance, autocorrelation, etc. are all constant over time.
#1-stabilize variance
df_logg = df.map(np.log)
#2-Removing Trend
df_logg_diff = df_logg

# Evaluating stationarity(Dickey-Fuller test )

In [ ]:
# Perform Dickey-Fuller test:Law pvalue mean the time series is stationary The p value must be less than alpha
from statsmodels.tsa.stattools import adfuller.diff()
adfstat, pvalue, usedlag, nobs, critvalues, icbest = adfuller(df_logg_diff.dropna())
